In [1]:
import numpy as np
import tensornetwork as tn
import tensorflow as tf

# Layers

In [14]:
class QuantumDMRGLayer(tf.keras.layers.Layer):
    def __init__(self, dimvec, pos_label, nblabels, bond_len, nearzero_std=1e-9):
        super(QuantumDMRGLayer, self).__init__()
        self.dimvec = dimvec
        self.pos_label = pos_label
        self.nblabels = nblabels
        self.m = bond_len
        
        assert self.pos_label >= 0 and self.pos_label < self.dimvec

        self.mps_tensors = [tf.Variable(self.mps_tensor_initial_values(i, nearzero_std=nearzero_std),
                                        trainable=True,
                                        name='mps_tensors_{}'.format(i))
                            for i in range(self.dimvec)]
        self.output_tensor = tf.Variable(tf.random.normal((self.m, self.m, self.nblabels),
                                                          mean=0.0,
                                                          stddev=nearzero_std),
                                         trainable=True,
                                         name='mps_output_node')

    def mps_tensor_initial_values(self, idx, nearzero_std=1e-9):
        if idx == 0 or idx == self.dimvec-1:
            tempmat = tf.eye(max(2, self.m))
            mat = tempmat[0:2, :] if 2 < self.m else tempmat[:, 0:self.m]
            return mat + tf.random.normal(mat.shape, mean=0.0, stddev=nearzero_std)
#         elif idx == self.pos_label:
#             return tf.random.normal((2, self.m, self.m, self.nblabels),
#                                     mean=0.0,
#                                     stddev=nearzero_std)
        else:
            return tf.random.normal((2, self.m, self.m),
                                    mean=0.0,
                                    stddev=nearzero_std)

    def infer_single(self, input):
        assert input.shape[0] == self.dimvec
        assert input.shape[1] == 2
        
        nodes = [
            tn.Node(self.mps_tensors[i], backend='tensorflow')
            for i in range(self.dimvec)
        ]
        output_node = tn.Node(self.output_tensor, backend='tensorflow')
        input_nodes = [
            tn.Node(input[i, :], backend='tensorflow')
            for i in range(self.dimvec)
        ]

        for i in range(self.dimvec):
            nodes[i][0] ^ input_nodes[i][0]
        nodes[0][1] ^ nodes[1][1]
        for i in range(1, self.pos_label):
            nodes[i][2] ^ nodes[i+1][1]
        nodes[self.pos_label][2] ^ output_node[0]
        output_node[1] ^ nodes[self.pos_label+1][1]
        for i in range(self.pos_label+1, self.dimvec-1):
            nodes[i][2] ^ nodes[i+1][1]

        final_node = tn.contractors.auto(nodes + input_nodes + [output_node],
                                         output_edge_order=[output_node[2]])
        return final_node.tensor

    def call(self, inputs):
        return tf.vectorized_map(self.infer_single, inputs)


In [15]:
quantum_dmrg_model = tf.keras.Sequential([
    tf.keras.Input(shape=(748, 2)),
    QuantumDMRGLayer(dimvec=748, pos_label=392, nblabels=10, bond_len=5),
    tf.keras.layers.Softmax()
])
quantum_dmrg_model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy())

In [16]:
quantum_dmrg_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantum_dmrg_layer_6 (Quantu (None, 10)                37570     
_________________________________________________________________
softmax_5 (Softmax)          (None, 10)                0         
Total params: 37,570
Trainable params: 37,570
Non-trainable params: 0
_________________________________________________________________


In [17]:
cosx = np.random.uniform(size=748)

inputs = np.array([np.array([np.cos(0.5*np.pi*cosx/256.), np.sin(0.5*np.pi*cosx/256.)]).T])

In [18]:
inputs

array([[[9.99981307e-01, 6.11430656e-03],
        [9.99999186e-01, 1.27628900e-03],
        [9.99992396e-01, 3.89975007e-03],
        ...,
        [9.99994360e-01, 3.35869069e-03],
        [9.99989856e-01, 4.50420141e-03],
        [9.99999996e-01, 8.88449102e-05]]])

In [19]:
output = quantum_dmrg_model.predict(inputs)
output

array([[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]], dtype=float32)

In [21]:
quantum_dmrg_model.layers[0].trainable_weights

[<tf.Variable 'mps_tensors_0:0' shape=(2, 5) dtype=float32, numpy=
 array([[ 1.0000000e+00, -1.0984637e-09, -8.6782509e-10,  7.1434712e-11,
         -1.7754422e-10],
        [-1.7623165e-09,  1.0000000e+00, -7.6668949e-10,  9.0300500e-10,
          7.1262923e-10]], dtype=float32)>,
 <tf.Variable 'mps_tensors_1:0' shape=(2, 5, 5) dtype=float32, numpy=
 array([[[-6.0549332e-10, -7.3542200e-11,  6.3614680e-10, -2.8020386e-09,
          -1.3226514e-09],
         [ 1.2170508e-09, -1.4335823e-09,  7.9236950e-10,  3.4409620e-10,
          -1.4608058e-09],
         [ 5.6257199e-10,  2.0492676e-09,  2.7517766e-10, -2.3065696e-09,
           5.5418125e-11],
         [-1.0060882e-09, -2.3812943e-11,  2.6225572e-10,  1.1470306e-09,
           1.1068890e-09],
         [-3.7009454e-10,  9.2799557e-10, -8.1220930e-10,  8.9532454e-10,
           5.7350219e-10]],
 
        [[-4.8484439e-10,  7.8537499e-10,  7.5950923e-10,  1.0507212e-09,
           1.3485372e-09],
         [ 6.4170069e-10, -3.1904909e-